In [1]:
import pandas as pd
import os
import re
import matplotlib.pyplot as plt

In [2]:
links = ['http://dati.ustat.miur.it/dataset/a60a221d-1c0d-4abb-bc8b-2199f61c205d/resource/0f69cde4-567f-468b-8bc1-fc861ac26590/download/bdg_serie_iscritti.csv',
        'http://dati.ustat.miur.it/dataset/a60a221d-1c0d-4abb-bc8b-2199f61c205d/resource/3098c012-08de-4085-b532-66c00e72a6cf/download/bdg_serie_laureati.csv',
        'http://dati.ustat.miur.it/dataset/a60a221d-1c0d-4abb-bc8b-2199f61c205d/resource/74fc14c9-571f-4181-888e-ef17e014793e/download/bdg_serie_dottorandi.csv',
        'http://dati.ustat.miur.it/dataset/a60a221d-1c0d-4abb-bc8b-2199f61c205d/resource/155998a7-c183-4eb3-b8cd-ca4d7db72b64/download/bdg_serie_dottori.csv']

In [3]:
def leggi_csv(link):
    df = pd.read_csv(link,encoding = "ISO-8859-1",sep=';')
    df = df.rename(columns={
        df.columns[-1]:'Numero'
    })
    categoria = os.path.split(link)[-1][:-4].split('_')[-1]
    df['categoria'] = [categoria]*len(df)
    return df

In [4]:
dfs = [leggi_csv(x) for x in links]

In [5]:
stud = pd.concat(dfs,join='inner')

In [6]:
stud = stud[stud['CorsoTIPO']!='Vecchio Ordinamento'] #leviamo i vecchi ordinamenti

In [7]:
stud['ANNO'].unique() #uniformiamo tutto sull'anno a sinistra in quelli con formato x/x+1

array(['2021/2022', '2020/2021', '2019/2020', '2018/2019', '2017/2018',
       '2016/2017', '2015/2016', '2014/2015', '2013/2014', '2012/2013',
       2021, 2020, 2019, 2018, 2017, 2016, 2015, 2014, 2013, 2012],
      dtype=object)

In [8]:
diz = dict(zip(stud['ANNO'].unique()[0:10],stud['ANNO'].unique()[10:])) #mappiamo vecchi valori a nuovi

In [9]:
stud['ANNO'].loc[:] = stud['ANNO'].apply(lambda x: diz.get(x, x)).astype(int)

In [10]:
prof = pd.read_csv('http://dati.ustat.miur.it/dataset/a60a221d-1c0d-4abb-bc8b-2199f61c205d/resource/92f2008d-958f-4e9c-ae5c-7a3dd418cd57/download/bdg_serie_academic_staff_ambito.csv',encoding = "ISO-8859-1", delimiter=';')

In [11]:
stud.columns

Index(['ANNO', 'AteneoCOD', 'AteneoNOME', 'AteneoREGIONE', 'AteneoAREAGEO',
       'CorsoTIPO', 'COD_FoET2013', 'DESC_FoET2013', 'Genere', 'Numero',
       'categoria'],
      dtype='object')

In [12]:
prof = prof.rename(columns={
    'CODICE_ATENEO':'AteneoCOD',
    'NOME_ATENEO':'AteneoNOME',
    'REGIONE':'AteneoREGIONE',
    'AREA_GEO': 'AteneoAREAGEO',
    'GENERE':'Genere',
    'N_AcStaff':'Numero',
    'GRADE':'categoria'
})

In [13]:
decod = pd.read_csv('http://dati.ustat.miur.it/dataset/a60a221d-1c0d-4abb-bc8b-2199f61c205d/resource/17b34084-9c01-4e90-9260-982fcb982e6a/download/cod_grade.csv', encoding = "ISO-8859-1", delimiter=';')

In [14]:
decod_grade = dict(zip(decod['GRADE'],decod['Qualifica']))

In [15]:
decod_grade['C'] = 'Ricercatore'

In [17]:
prof['categoria'] = prof['categoria'].apply(lambda x: decod_grade[x])

In [18]:
#troviamo per materie prof, se è stem o no!
materie_prof = pd.read_csv('http://dati.ustat.miur.it/dataset/a60a221d-1c0d-4abb-bc8b-2199f61c205d/resource/688a0ad5-a4f7-469d-af3d-f7448aa5ae30/download/cod_ford.csv',encoding = "ISO-8859-1", delimiter=';')
decoder_prof = dict(zip(materie_prof['FoRD'],materie_prof['Area STEM']))

In [19]:
prof['STEM'] = prof['FoRD'].apply(lambda x: decoder_prof[x])

In [20]:
#ora stessa cosa per stud
materie_stud = pd.read_csv('http://dati.ustat.miur.it/dataset/a60a221d-1c0d-4abb-bc8b-2199f61c205d/resource/3f52db2f-24ce-4605-8e51-5618cc4ff4e3/download/cod_foet2013.csv',encoding = "ISO-8859-1", delimiter=';')
decoder_stud = dict(zip(materie_stud['ISCED_F_1dgt'],materie_stud['Area STEM']))

In [21]:
stud.sample()

,ANNO,AteneoCOD,AteneoNOME,AteneoREGIONE,AteneoAREAGEO,CorsoTIPO,COD_FoET2013,DESC_FoET2013,Genere,Numero,categoria
26902,2012,04101,"Urbino - Università degli studi ""Carlo Bo""",Marche,CENTRO,Laurea,10,Services,M,128,laureati


In [22]:
stud['STEM'] = stud['COD_FoET2013'].apply(lambda x: decoder_stud[x])

In [23]:
df = pd.concat([stud,prof],join='inner') #adesso sappiamo se sono STEM o no!

In [42]:
df['STEM'] = df['STEM'].replace('No',False).replace('Sì',True).astype(bool) #trasformiamo in booleani

In [329]:
def conto(df, relative=False, flourish=False): 
    '''
    
    
    
    big funzione che serve a fare i conti
    
    
    '''
    
    df = df.groupby(['categoria','Genere']).agg({
        'Numero':pd.Series.sum
    }).reset_index()
    
    def ordina(colonna): #funzione di supporto solo per averli in ordine 
        diz = {
            'iscritti':0,
            'laureati':1,
            'dottorandi':2,
            'dottori':3,
            'Assegnista di Ricerca':4,
            'Ricercatore':5,
            'Prof. Associato':6,
            'Prof. Ordinario':7
        }
        colonna = colonna.apply(lambda x:diz[x])
        return colonna
    
    df = df.sort_values(by=['categoria'],key=ordina)
    
    if flourish:
        return df
    
    M= []
    F=[]
    colons = []

    for cat in diz:
        try:
            M.append(df[(df['categoria']==cat)&(df['Genere']=='M')]['Numero'].iloc[0])
        except IndexError as e:
            print(f'M - {cat} ---> {e}')
            M.append(0)
        
        try:
            F.append(df[(df['categoria']==cat)&(df['Genere']=='F')]['Numero'].iloc[0])
            
        except IndexError as e:
            print(f'F - {cat} ---> {e}')
            F.append(0)
            
        colons.append(cat)
    
    ndf = pd.DataFrame([M,F],columns=colons,index=['M','F'])
    if relative:
        ndf = ndf.apply(lambda x: x/x.sum()*100)
        
    return ndf

pd.DataFrame.conto = conto

In [220]:
def filtro(df, nome, stem=None, anno=2021): 
    '''
    
    funzione per filtrare per università, STEM (False,True,None), anno
    
    
    '''
    
    if stem is not None:
        df = df[df['STEM']==stem]
        
    return df[(df['AteneoNOME']==nome)&(df['ANNO']==anno)]
pd.DataFrame.filtro = filtro

In [232]:
def grafico(df,transpose=True,tit=None, relative=True):
    
    '''
    
    funzione per fare il grafichetto
    
    '''
    if relative:
        df = df.apply(lambda x: x/x.sum()*100)
    
    if transpose:
        df = df.transpose()
    
    fig,ax = plt.subplots(figsize=(14,6))
    plt.ylim(0,100)
    plt.plot(df['M'],'o-', label='maschi')
    plt.plot(df['F'],'o-',label='femmine')
    plt.legend()
    
    if tit:
        ax.set_title(tit)

    for x,y_m,y_f in zip(df.index,df['M'],df['F']):
        plt.annotate(f'{round(y_m,1)}%', # this is the text
                         (x,y_m), # these are the coordinates to position the label
                         textcoords="offset points", # how to position the text
                         xytext=(0,8), # distance from text to points (x,y)
                         ha='center',
                         size=10,
                         style='italic')

        plt.annotate(f'{round(y_f,1)}%', # this is the text
                         (x,y_f), # these are the coordinates to position the label
                         textcoords="offset points", # how to position the text
                         xytext=(0,-13), # distance from text to points (x,y)
                         ha='center',
                         size=10,
                         style='italic')
    return fig

pd.DataFrame.grafico = grafico

In [233]:
def indice(df, considera_studenti=False):
    '''
    Glass Ceiling Index definito dall'UE
    
    1 = donne hanno le stesse chance degli uomini di essere promosse
    0 = donne sono più rappresentate al grado massimo rispetto a uomini
    >1 = uomini più rappresentati
    
    '''
    rep_tot_femm = df.loc['F'].iloc[0 if considera_studenti else 5:].sum()
    rep_tot = rep_tot_femm + df.loc['M'].iloc[0 if considera_studenti else 5:].sum()
    rep_ceil_femm = df.loc['F'].iloc[-1]
    rep_ceil = rep_ceil_femm + df.loc['M'].iloc[-1]
    
    i = (rep_tot_femm/rep_tot) / (rep_ceil_femm/rep_ceil)
    
    return i

pd.DataFrame.indice = indice

In [304]:
uni = 'TOTALE ATENEI'
ndf = df.filtro(uni,stem=False).conto()

In [320]:
ndf.grafico(tit=f'{uni}').savefig()

TypeError: savefig() missing 1 required positional argument: 'fname'

In [325]:
for ateneo in df['AteneoNOME'].unique():
    
    ndf = df.filtro(ateneo, stem=False).conto()
    if 'iscritti' in ndf.columns and ndf['iscritti'].sum() > 15000:
        path = f'/Users/sava/Desktop/Università genere/{ateneo}/NO_STEM'
        os.makedirs(path,exist_ok=True)
        ndf.to_csv(f'{path}/no_stem.csv')
        fig = ndf.grafico(tit=f'{ateneo} - no stem')
        fig.savefig(f'{path}/no_stem.png')
    
    ndf = df.filtro(ateneo, stem=True).conto()
    
    if 'iscritti' in ndf.columns and ndf['iscritti'].sum() > 15000:
        path = f'/Users/sava/Desktop/Università genere/{ateneo}/STEM'
        os.makedirs(path,exist_ok=True)
        ndf.to_csv(f'{path}/stem.csv')
        ndf.grafico(tit=f'{ateneo} - stem')
        fig = ndf.grafico(tit=f'{ateneo} - no stem')
        fig.savefig(f'{path}/stem.png')

M - Assegnista di Ricerca ---> single positional indexer is out-of-bounds
F - Assegnista di Ricerca ---> single positional indexer is out-of-bounds
M - Assegnista di Ricerca ---> single positional indexer is out-of-bounds
F - Prof. Ordinario ---> single positional indexer is out-of-bounds
M - Ricercatore ---> single positional indexer is out-of-bounds
F - Prof. Ordinario ---> single positional indexer is out-of-bounds
F - Assegnista di Ricerca ---> single positional indexer is out-of-bounds
F - Assegnista di Ricerca ---> single positional indexer is out-of-bounds
M - Ricercatore ---> single positional indexer is out-of-bounds
M - dottori ---> single positional indexer is out-of-bounds
M - Assegnista di Ricerca ---> single positional indexer is out-of-bounds
M - Prof. Associato ---> single positional indexer is out-of-bounds
F - Prof. Ordinario ---> single positional indexer is out-of-bounds
F - Assegnista di Ricerca ---> single positional indexer is out-of-bounds
F - Prof. Ordinario --

In [339]:
df['AteneoREGIONE'].unique()

array(['ITALIA', 'Sardegna', 'Sicilia', 'Calabria', 'Basilicata',
       'Puglia', 'Molise', 'Abruzzo', 'Campania', 'Lazio', 'Umbria',
       'Toscana', 'Marche', 'Emilia-Romagna', 'Friuli-Venezia Giulia',
       'Veneto', 'Trentino-Alto Adige', 'Lombardia', 'Liguria',
       "Valle d'Aosta", 'Piemonte'], dtype=object)